In [1]:
import tensorflow as tf
import time
def curTime():
    return round(time.time() * 1000)
print(28500 % 150)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4129374754674355753
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6290407424
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3238188444203119072
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2070, pci bus id: 0000:29:00.0, compute capability: 7.5"
]
2.6.2
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# Extracting datasets to input matrix
import numpy as np
import pandas as pd
from tqdm import tqdm
import random



dataset_size = 28000
pred_dataset_size = 7500

flow_size = 150
number_flows = int(dataset_size/flow_size) -2 #substract 2 so first two flows are ignored
pred_number_flows = int(pred_dataset_size/flow_size) - 2
seed=2
val=0 # For spliting number of flows for training .csv in training and test
# paths
watermarked_flows = pd.read_csv('caps/TirSkypeTrainBuf50Icbw120.csv', usecols=['Time']) # 28787
regular_flows = pd.read_csv('caps/TirSkypeTrainBuf50none.csv', usecols=['Time']) # 45000
watermarked_flows_prediction = pd.read_csv('caps/TirSkypeTestBuf50Icbw120.csv', usecols=['Time']) # 10109
regular_flows_prediction = pd.read_csv('caps/TirSkypeTestBuf50none.csv', usecols=['Time']) # 8981



features = np.zeros((number_flows*2, 1, flow_size, 1))
classes = np.zeros(features.shape[0])
indexes = np.arange(features.shape[0])

# random.Random(seed).shuffle(indexes) # Consistent randomization
random.Random(3).shuffle(indexes) # Inconsistent randomization

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=4, shuffle=True)
for train_indexes,valid_indexes in skf.split(features, classes):
    break
    
print((train_indexes.shape, valid_indexes.shape))    
    

def outlier(delay):
    return delay if delay < 5000 else 100

    
watermarked_index = flow_size;
regular_index = flow_size;
for flow_index in tqdm(range(0, number_flows*2)):      #changed range to start from 0 so first two flows are ignored
    if flow_index%2==0:
        classes[flow_index]=1
        for i in range(flow_size):  
            features[flow_index,0,i]=outlier((watermarked_flows['Time'][watermarked_index]-watermarked_flows['Time'][watermarked_index-1])*1000)
            watermarked_index+=1
    else:
        classes[flow_index]=0
        for i in range(flow_size):
            features[flow_index,0,i]=outlier((regular_flows['Time'][regular_index]-regular_flows['Time'][regular_index-1])*1000)
            regular_index+=1
    

print(features.shape)

((276,), (92,))


100%|██████████| 368/368 [00:00<00:00, 686.98it/s]

(368, 1, 150, 1)


In [3]:
#         ================================================================================================
#         ======================================== Model of the CNN ======================================
#         ================================================================================================


import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Layer, Dense, Conv2D, MaxPool2D, Dropout, Flatten,
                                     GlobalAveragePooling2D, ZeroPadding2D)


kernel_size = ([1, 10], [1, 10])
filters = (500, 100)

model = Sequential()
model.add(Conv2D(filters=filters[0], kernel_size=kernel_size[0], strides=(1,1), activation='relu'))
model.add(MaxPool2D([1,5]))
model.add(Conv2D(filters=filters[1], kernel_size=kernel_size[1], strides=(1,1), activation='relu'))
model.add(MaxPool2D([1,5]))
model.add(Flatten())

model.add(Dense(units=3000, activation='relu'))
model.add(Dropout(0.6))

model.add(Dense(units=800, activation='relu'))
model.add(Dropout(0.6))

model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
#         ================================================================================================
#         ======================================== Training ==============================================
#         ================================================================================================


from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
learning_rate = 0.0001
batch_size=256
epochs=150
loss = 'binary_crossentropy'
callback = EarlyStopping(monitor='val_loss', patience = 50)

optimizer=Adam(learning_rate=learning_rate)
model.compile(optimizer = optimizer, loss=loss, metrics=['binary_accuracy'])
timestamp1 = curTime()
history = model.fit(features[train_indexes[val:]], classes[train_indexes[val:]], validation_data=(features[valid_indexes],classes[valid_indexes]), batch_size=batch_size, epochs=epochs, callbacks=[callback])

In [5]:
#         ================================================================================================
#         ======================================== Testing ==============================================
#         ================================================================================================

timestamp2 = curTime()

features_prediction = np.zeros((pred_number_flows*2, 1, flow_size, 1))
classes_prediction = np.zeros(features_prediction.shape[0])

pred_watermarked_index = flow_size;
pred_regular_index = flow_size;
for flow_index in tqdm(range(0, pred_number_flows*2)):      #changed range to start from 0 so first two flows are ignored
    if flow_index%2==0:
        classes_prediction[flow_index]=1
        for i in range(flow_size):  
            features_prediction[flow_index,0,i]=outlier((watermarked_flows_prediction['Time'][pred_watermarked_index]-watermarked_flows_prediction['Time'][pred_watermarked_index-1])*1000)
            pred_watermarked_index+=1
    else:
        classes_prediction[flow_index]=0
        for i in range(flow_size):
            features_prediction[flow_index,0,i]=outlier((regular_flows_prediction['Time'][pred_regular_index]-regular_flows_prediction['Time'][pred_regular_index-1])*1000)
            pred_regular_index+=1


fp=0; fn=0; tp=0; tn=0
pred_classes = classes_prediction
timestamp3 = curTime()
pred = model.predict(features_prediction)
timestamp4 = curTime()
print(('TEST',classes[train_indexes].shape[0]-val, classes[valid_indexes].shape, pred_classes.shape))

#print(pred_classes)
#print(np.around(pred.flatten(),3))
for i in range(pred.shape[0]):
    if pred_classes[i]==0:
        if pred[i]>0.5: fp+=1
        else: tn+=1
    else:
        if pred[i]>0.5: tp+=1
        else: fn+=1            

# Sensitivity, hit rate, recall, or true positive rate
tpr = tp/(tp+fn)
# Fall out or false positive rate
fpr = fp/(fp+tn)
metrics = [fpr, tpr, (tp+tn)/(tn+tp+fp+fn)]
print("FPS|TPR|BINARY-ACC")
print(np.around(metrics, 4))





100%|██████████| 96/96 [00:00<00:00, 676.58it/s]


('TEST', 276, (92,), (96,))
FPS|TPR|BINARY-ACC
[0.0417 0.7292 0.8438]


In [6]:
#print(('TEST',kernel_size,classes[train_indexes].shape[0]-val, classes[valid_indexes].shape, pred_classes.shape))
print('TEST FPS|TPR|BINARY-ACC')
results = np.around(metrics,4)
listToStr = ' '.join([str(elem) for elem in results])
print(listToStr.replace(".",","))
fp=0; fn=0; tp=0; tn=0    
if(True):
    pred_classes = classes[valid_indexes]
    pred = model.predict(features[valid_indexes])
else:
    pred_classes = classes[train_indexes]
    pred = model.predict(features[train_indexes])
#print(pred_classes)
#print(np.around(pred.flatten(),3))
for i in range(pred.shape[0]):
    if pred_classes[i]==0:
        if pred[i]>0.5: fp+=1
        else: tn+=1
    else:
        if pred[i]>0.5: tp+=1
        else: fn+=1            
# Sensitivity, hit rate, recall, or true positive rate
tpr = tp/(tp+fn)
# Fall out or false positive rate
fpr = fp/(fp+tn)
metrics = [fpr, tpr, (tp+tn)/(tn+tp+fp+fn)]
print("\nVal - FPS|TPR|BINARY-ACC")
results = np.around(metrics,4)
listToStr = ' '.join([str(elem) for elem in results])
print(listToStr.replace(".",","))

print((timestamp2-timestamp1,timestamp4-timestamp3))

TEST FPS|TPR|BINARY-ACC
0,0417 0,7292 0,8438

Val - FPS|TPR|BINARY-ACC
0,0244 1,0 0,9891
(8595, 116)


In [7]:
#         ================================================================================================
#         =================================  Testing with same circuits ==================================
#         ================================================================================================

if val!=0:
    fp=0; fn=0; tp=0; tn=0
    pred_classes = classes[train_indexes[:val]]
    pred = model.predict(features[train_indexes[:val]])
    print((kernel_size, classes[train_indexes[:-val]].shape, classes[valid_indexes].shape, pred_classes.shape))
    #print(pred_classes)
    #print(np.around(pred.flatten(),3))
    for i in range(pred.shape[0]):
        if pred_classes[i]==0:
            if pred[i]>0.5: fp+=1
            else: tn+=1
        else:
            if pred[i]>0.5: tp+=1
            else: fn+=1            

    # Sensitivity, hit rate, recall, or true positive rate
    tpr = tp/(tp+fn)
    # Fall out or false positive rate
    fpr = fp/(fp+tn)
    metrics = [fpr, tpr, (tp+tn)/(tn+tp+fp+fn)]
    print("FPS|TPR|BINARY-ACC")
    results = np.around(metrics, 4) 
    listToStr = ' '.join([str(elem) for elem in results])
    print(listToStr.replace(".",","))
    
    